In [1]:
import numpy as np
import pandas as pd
import re
import torch
import torch.nn as nn
import torchtext
import torecsys as trs

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1

In [2]:
# get samples data from movielens as a example
# _, movies_df, ratings_df, _ = trs.data.sampledata.load_ml_data(size="latest-small")
# movies_df["year"] = movies_df.title.apply(lambda x: re.findall(r"\((\d+)\)", x))
# movies_df["year"] = movies_df.year.apply(lambda x: int(x[0]) if len(x) > 0 else np.nan)
# movies_df = pd.concat([
#     movies_df, 
#     pd.get_dummies(movies_df.genres.apply(
#         lambda x: x.split("|")).apply(pd.Series).stack()).sum(level=0)
# ], axis=1).drop(["title", "genres"], axis=1)
# merged = pd.merge(ratings_df, movies_df, on="movieId")

# Inputs

In [3]:
emb_dict = torch.jit.trace(trs.inputs.base.EmbeddingDict(["userId", "itemId"], ["single_index", "single_index"], [100, 50], [16, 16]), {"userId": (torch.Tensor([[1]]).long()), "itemId": (torch.Tensor([[1]]).long())})
emb_dict({"userId": (torch.Tensor([[1]]).long()), "itemId": (torch.Tensor([[1]]).long())})

/home/jasper-lwy430/Documents/git/torecsys/torecsys/utils/decorator/decorator.py:49: UserWarning: the module have been checked with torch.jit.trace, but the feature is in experimemtal.!!! Remark. It is not allow to use dropout at this stage, since inferences of the same inputs will be different after dropout layer. So, Please use dropout_p = 0.0 or None.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/torch/tensor.py:389: RuntimeWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  'incorrect results).', category=RuntimeWarning)
/home/jasper-lwy430/Documents/git/torecsys/torecsys/inputs/base/emb_dict.py:98: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This m

tensor([[[-0.1912, -0.6230, -0.8830, -0.2447,  0.5507, -0.3032,  1.2818,
          -0.5683, -0.4514,  1.9029,  0.4188, -0.2563, -0.4712, -1.1331,
          -1.2220, -1.0928],
         [ 0.9362,  0.2546,  1.2643, -0.8709,  0.5111, -0.8296, -1.9107,
           0.9402, -0.0376, -0.5970, -1.7095,  0.2658,  0.6568, -1.2178,
          -2.1297, -0.9016]]], grad_fn=<DifferentiableGraphBackward>)

In [ ]:
fa_emb = torch.jit.trace(trs.inputs.base.FieldAwareMultipleIndexEmbedding(16, [100, 10]), torch.Tensor([[1, 1]]).long())
fa_emb(torch.Tensor([[1, 1]]).long())

In [ ]:
list_emb = torch.jit.trace(trs.inputs.base.ListIndexEmbedding(16, 100, num_heads=4), torch.Tensor([[1, 4, 5]]).long())
list_emb(torch.Tensor([[1, 4, 5]]).long())
# try to fix if use trace
# list_emb.show_attention(torch.Tensor([[1, 4, 5]]).long())

In [ ]:
mlp_emb = torch.jit.trace(trs.inputs.base.MultipleIndexEmbedding(16, [100, 10]), torch.Tensor([[1, 1]]).long())
mlp_emb(torch.Tensor([[1, 1]]).long())

In [ ]:
sq_emb = torch.jit.trace(trs.inputs.base.SequenceIndexEmbedding(16, 100), (torch.Tensor([[3, 1, 0], [4, 2, 5]]).long(), torch.Tensor([2, 3]).long()))
sq_emb(torch.Tensor([[3, 1, 0], [4, 2, 5]]).long(), torch.Tensor([2, 3]).long())

In [ ]:
index_emb = torch.jit.trace(trs.inputs.base.SingleIndexEmbedding(16, 100), torch.Tensor([[1]]).long())
index_emb(torch.Tensor([[1]]).long())

In [6]:
stack = torch.jit.trace(trs.inputs.base.StackedInputs(["userId", "itemId"], ["single_index", "single_index"], [100, 50], [16, 16]), ({"userId": (torch.Tensor([[1]]).long()), "itemId": (torch.Tensor([[1]]).long())}))
stack({"userId": (torch.Tensor([[1]]).long()), "itemId": (torch.Tensor([[1]]).long())})

/home/jasper-lwy430/Documents/git/torecsys/torecsys/inputs/base/stacked_inp.py:88: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if not all(field_input.size(0) == batch_size for field_inputs in inputs.values() for field_input in field_inputs):
/home/jasper-lwy430/Documents/git/torecsys/torecsys/inputs/base/stacked_inp.py:94: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  outputs.append(self.embeddings[fname](*inputs[fname]))


In [13]:
val = torch.jit.trace(trs.inputs.base.ValueInputs(5), torch.randn(size=(1, 5)))
val(torch.randn(size=(1, 5)))

tensor([[[ 0.2867,  0.3543, -0.2745,  0.1554, -0.8108]]])

In [ ]:
e = 8
n = 5
b = 16

feat_inputs = torch.randn(size=(b, n, 1))
field_emb = torch.randn(size=(b, n, e))
field_aware_emb = torch.randn(size=(b, n * n, e))

In [ ]:
user_item_emb = torch.randn(size=(b, 2, e))

# Models

In [ ]:
afm_model = torch.jit.trace(trs.models.AttentionalFactorizationMachineModel(e, n, 8, 0.0), (feat_inputs, field_emb))
afm_model(feat_inputs, field_emb)

In [ ]:
dc_model = torch.jit.trace(trs.models.DeepAndCrossNetworkModel(e * n, 1, [10], 2, 1, [0.0], nn.ReLU6()), field_emb)
dc_model(field_emb)

In [ ]:
dffm_model = torch.jit.trace(trs.models.DeepFieldAwareFactorizationMachineModel(e, n, 1, [10], 0.0, [0.0], nn.Tanh(), 1), field_aware_emb)
dffm_model(field_aware_emb)

In [ ]:
dfm_model = torch.jit.trace(trs.models.DeepFactorizationMachineModel(e, n, [10], 0.0, [0.0], nn.ReLU(), 1), (feat_inputs, field_emb))
dfm_model(feat_inputs, field_emb)

In [ ]:
fm = torch.jit.trace(trs.models.FactorizationMachineModel(e, n, 0.1), (feat_inputs, field_emb))
fm(feat_inputs, field_emb)

In [ ]:
fmnn = torch.jit.trace(trs.models.FactorizationMachineSupportedNeuralNetwork(e, n, 1, [10], 0.1, [0.1], nn.Tanh()), (feat_inputs, field_emb))
fmnn(feat_inputs, field_emb)

In [ ]:
fatdffm = torch.jit.trace(trs.models.FieldAttentiveDeepFieldAwareFactorizationMachineModel(e, n, 10, [10], 16, 0.0, [0.1], nn.Tanh()), field_aware_emb)
fatdffm(field_aware_emb)

In [ ]:
ffm = torch.jit.trace(trs.models.FieldAwareFactorizationMachineModel(e, n), (feat_inputs, field_aware_emb))
ffm(feat_inputs, field_aware_emb)

In [ ]:
lr = torch.jit.trace(trs.models.LogisticRegressionModel(n), feat_inputs)
lr(feat_inputs)

In [ ]:
ncf = torch.jit.trace(trs.models.NeuralCollaborativeFilteringModel(e, 1, [10], [0.1], nn.ReLU6()), user_item_emb)
ncf(user_item_emb)

# Layers

In [ ]:
afm = trs.layers.AttentionalFactorizationMachineLayer(e, n, attn_size=8, dropout_p=0.0)
print(afm(field_emb)[0].size())
print(afm(field_emb)[1].size())

In [ ]:
blin = trs.layers.BilinearNetworkLayer(e, n, 1, 3)
blin(field_emb).size()

In [ ]:
cen = trs.layers.ComposeExcitationNetworkLayer(n)
cen(field_aware_emb).size()

In [ ]:
cin = trs.layers.CompressInteractionNetworkLayer(e, n, 1, [10, 10, 10])
cin(field_emb).size()

In [ ]:
cross = trs.layers.CrossNetworkLayer(5, e, n)
cross(field_emb).size()

In [ ]:
fm = trs.layers.FactorizationMachineLayer(0.0)
fm(field_emb).size()

In [ ]:
ffm = trs.layers.FieldAwareFactorizationMachineLayer(n, 0.0)
ffm(field_aware_emb).size()

In [ ]:
inner = trs.layers.InnerProductNetworkLayer(n)
inner(field_emb).size()

In [ ]:
mlp = trs.layers.MultilayerPerceptronLayer(1, [10, 10, 10], embed_size=e, num_fields=n, dropout_p=[0.0, 0.0, 0.0])
mlp(field_emb).size()

In [ ]:
outer = trs.layers.OuterProductNetworkLayer(e, n)
outer(field_emb).size()

In [ ]:
wide = trs.layers.WideLayer(e, n, 1)
wide(field_emb).size()

In [ ]:
afm = trs.models.AttentionalFactorizationMachineModel(e, n, 8)